# Inputs
---

In [1]:
import numpy as np, zarr, nrrd
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
import tifffile
import seaborn as sns

In [2]:
p      = '/u/home/f/f7xiesnm/project-zipursky/easifish/cdf04_c1-2_bino/outputs/'
outdir = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream_cdf04_r3tor1_copy1'
!mkdir -p $outdir

In [3]:
fix_path = p + '/r1/stitching/export.n5'
mov_path = p + '/r3/stitching/export.n5'
exp_factor = 2  # replace this with the known expansion factor for your sample to use pre-expansion units

fix_spacing = np.array([0.42, 0.23, 0.23]) / exp_factor
mov_spacing = np.array([0.42, 0.23, 0.23]) / exp_factor
s1_scale = [1,2,2] # z,y,x
s3_scale = [4,8,8]
s4_scale = [8,16,16]

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
# fix_meta = fix_zarr.attrs.asdict()
# fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * s1_scale 
fix_spacing_s3 = fix_spacing * s3_scale 
fix_spacing_s4 = fix_spacing * s4_scale 

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
# mov_meta = mov_zarr.attrs.asdict()
# mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * s1_scale
mov_spacing_s3 = mov_spacing * s3_scale 
mov_spacing_s4 = mov_spacing * s4_scale 

# print spacings
print(fix_spacing, mov_spacing)
print(fix_spacing_s1, mov_spacing_s1)
print(fix_spacing_s3, mov_spacing_s3)
print(fix_spacing_s4, mov_spacing_s4)

# print shapes
print(fix_zarr['/c3_reg/s3'].shape, mov_zarr['/c3_reg/s3'].shape)
print(fix_zarr['/c3_reg/s4'].shape, mov_zarr['/c3_reg/s4'].shape)

# # write some channels
# nrrd.write(outdir+'/fix.nrrd', fix_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)
# nrrd.write(outdir+'/mov.nrrd', mov_zarr['/c3/s4'][...].transpose(2,1,0), compression_level=2)

[0.21  0.115 0.115] [0.21  0.115 0.115]
[0.21 0.23 0.23] [0.21 0.23 0.23]
[0.84 0.92 0.92] [0.84 0.92 0.92]
[1.68 1.84 1.84] [1.68 1.84 1.84]
(530, 901, 906) (544, 903, 905)
(265, 450, 453) (272, 451, 452)


# Alignment
---

### global

In [4]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

In [5]:
# get global alignment channels
fix = fix_zarr['/c3_reg/s4'][...]
mov = mov_zarr['/c3_reg/s4'][...] # [:,::-1,::-1]

# fix[:50] = 0
# fix[-20:] = 0
# mov[:40] = 0
# mov[-20:] = 0
fix[:100] = 0
fix[-100:] = 0
mov[:100] = 0
mov[-100:] = 0

# # create some masks to focus alignment only on planes below the large artifact at the top
# fix_mask = np.ones(fix.shape, dtype=np.uint8)
# fix_mask[:50] = 0
# fix_mask[-20:] = 0

# mov_mask = np.ones(mov.shape, dtype=np.uint8)
# mov_mask[:40] = 0
# mov_mask[-20:] = 0

In [6]:
tifffile.imwrite(outdir+'/fix.tiff', fix)
tifffile.imwrite(outdir+'/mov.tiff', mov)
# tifffile.imwrite(outdir+'/mov0.tiff', mov0)

In [7]:

%%time

# define alignment steps
ransac_kwargs = {
    'blob_sizes': [2, 8],
    'cc_radius': 12,
    'match_threshold': 0.4, #0.6, # 0.4
    'nspots':10000,
    'fix_spot_detection_kwargs': {
            'threshold':5e-4,
            'threshold_rel':None,
        },
    'mov_spot_detection_kwargs': {
            'threshold':1e-3,
            'threshold_rel':None,
        },
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}


steps = [
    ('ransac', ransac_kwargs,),
    ('affine', affine_kwargs,),
        ]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    # fix_mask=fix_mask,
    # mov_mask=mov_mask,
)
print(affine)



# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt(outdir+'/affine-masked.mat', affine)
tifffile.imwrite(outdir+'/affine-masked.tiff', affine_aligned)

computing fixed spots
found 14999 fixed spots
computing moving spots
found 14588 moving spots
sorting spots
extracting contexts
computing pairwise correlations
2 matched spots
insufficient point matches found, returning default
LEVEL:  0  ITERATION:  0  METRIC:  -0.2048981482460222
LEVEL:  0  ITERATION:  1  METRIC:  -0.20885961112180795
LEVEL:  0  ITERATION:  2  METRIC:  -0.21229764354183556
LEVEL:  0  ITERATION:  3  METRIC:  -0.2151943912310682
LEVEL:  0  ITERATION:  4  METRIC:  -0.21753711189757097
LEVEL:  0  ITERATION:  5  METRIC:  -0.2193174340534921
LEVEL:  0  ITERATION:  6  METRIC:  -0.22053076520273138
LEVEL:  0  ITERATION:  7  METRIC:  -0.22115717888147463
LEVEL:  0  ITERATION:  8  METRIC:  -0.22113065019661876
LEVEL:  0  ITERATION:  9  METRIC:  -0.22055552471333187
LEVEL:  0  ITERATION:  10  METRIC:  -0.2195362947687907
LEVEL:  0  ITERATION:  11  METRIC:  -0.21894712087936477
LEVEL:  0  ITERATION:  12  METRIC:  -0.21931718877534098
LEVEL:  0  ITERATION:  13  METRIC:  -0.220167